# Test on Velodyne data

In [1]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go

from planeslam.general import downsample, adaptive_downsample
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters
from planeslam.extraction import scan_from_clusters, planes_from_clusters
from planeslam.general import pc_plot_trace
from planeslam.geometry.util import quat_to_R
from planeslam.point_cloud import velo_preprocess

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Flight room

In [2]:
# Read in data
frame_num = 310

pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'pcs')
filename = pcpath + '/pc_' + str(frame_num) + '.npy'
PC = np.load(filename)

posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'poses')
filename = posepath + '/pose_' + str(frame_num) + '.npy'
pose = np.load(filename)

In [3]:
fig = go.Figure(data=pc_plot_trace(PC))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [4]:
# Pre-process 
PC = velo_preprocess(PC, pose)
# Downsample
PC = adaptive_downsample(PC, factor=5)

mesh = LidarMesh(PC)
mesh.prune(edge_len_lim=0.5)

mesh.smooth_laplacian()
clusters, avg_normals = cluster_mesh_graph_search(mesh)
planes, basis = planes_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes, basis)

In [5]:
fig = go.Figure(data=[pc_plot_trace(PC)]+scan.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
fig = go.Figure(data=pc_plot_trace(PC))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
R = quat_to_R(pose[3:])
t = pose[:3]
PC = (R @ PC.T).T + t

In [ ]:
fig = go.Figure(data=pc_plot_trace(PC))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Pre-process 
PC = velo_preprocess(PC, pose)
# Downsample
PC = adaptive_downsample(PC, factor=5)

In [ ]:
fig = go.Figure(data=pc_plot_trace(PC))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Create the mesh
mesh = LidarMesh(PC)
mesh.prune(edge_len_lim=0.5)
# start_time = time.time()
# # Cluster the mesh with graph search
# clusters, avg_normals = cluster_mesh_graph_search(mesh)
# print("elapsed time: ", time.time() - start_time)

In [ ]:
import open3d as o3d

o3d_mesh = o3d.geometry.TriangleMesh()
o3d_mesh.vertices = o3d.utility.Vector3dVector(mesh.P)
o3d_mesh.triangles = o3d.utility.Vector3iVector(mesh.DT.simplices)

#o3d.visualization.draw_geometries([o3d_mesh])

In [ ]:
smoothed_mesh = o3d_mesh.filter_smooth_laplacian(number_of_iterations=1)

In [ ]:
mesh.P = np.asarray(smoothed_mesh.vertices)

In [ ]:
# Plot mesh
fig = go.Figure(data=mesh.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
clusters, avg_normals = cluster_mesh_graph_search(mesh)

In [ ]:
# Plot clusters
plot_clusters(PC, mesh, clusters)

In [ ]:
planes = planes_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes)

In [ ]:
fig = go.Figure(data=[pc_plot_trace(PC)]+scan.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
np.dot(avg_normals[0], avg_normals[2])

In [ ]:
from planeslam.general import normalize

v = avg_normals[0]
n = avg_normals[2][:,None]

v_proj = normalize(v - (v @ n) @ n.T)

In [ ]:
np.dot(v_proj, n)

In [ ]:
from planeslam.geometry.util import project_points_to_plane

n_proj = project_points_to_plane(avg_normals[0], avg_normals[2][:,None])
np.dot(n_proj, avg_normals[2])

In [ ]:
n_proj

In [ ]:
avg_normals[2][:,None]

In [ ]:
for p in scan.planes:
    print(p.normal)

In [ ]:
normals_arr[np.argmax(np.abs(normals_arr), axis=1)==2][0]

In [ ]:
from planeslam.scan import velo_pc_to_scan
scan = velo_pc_to_scan(PC, ds_rate=5, edge_len_lim=2)

In [ ]:
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
from planeslam.clustering import mesh_cluster_pts, sort_mesh_clusters

clusters, avg_normals = sort_mesh_clusters(clusters, avg_normals)
i = 0
n = avg_normals[i][:,None]
c = clusters[i]
cluster_pts = mesh_cluster_pts(mesh, c)  # Extract points from cluster

In [ ]:
n

In [ ]:
# Plot the points 
fig = go.Figure(data=pc_plot_trace(cluster_pts))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Find extraction basis based on normals
basis = np.zeros((3,3))
basis[:,2] = avg_normals[0]  # choose first cluster's normal as z
dps = np.asarray(avg_normals) @ basis[:,2]
orth_idxs = np.nonzero(np.abs(dps) < 0.2)[0]  # indices of normals approximately orthonormal to z
basis[:,0] = avg_normals[orth_idxs[0]]  # choose the first one as x
basis[:,1] = np.cross(basis[:,2], basis[:,0])

In [ ]:
basis

In [ ]:
basis.T @ basis